In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import lightgbm as lgb
import optuna.integration.lightgbm as olgb

import matplotlib.pyplot as plt
import seaborn as sns

***
## loading data

In [2]:
features = [f"f_{i}" for i in range(300)]

features = pd.read_parquet("../ump-dataset/train.parquet", columns=features)
display(features)

target = pd.read_parquet("../ump-dataset/train.parquet", columns=["target",])
display(target)

time = pd.read_parquet("../ump-dataset/train.parquet", columns=["time_id",])
display(time)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,0.965623,1.230508,0.114809,-2.012777,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,1.428127,1.134144,0.114809,-0.219201,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,0.979656,-1.125681,0.114809,-1.035376,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,0.778096,0.298990,0.114809,-1.176410,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,-0.946789,1.230508,0.114809,-0.005858,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,1.475746,1.284423,0.056425,-1.433681,...,-0.285908,-1.232434,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394
3141406,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,-1.498235,1.373834,0.056425,-1.211572,...,0.184517,-1.232434,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705
3141407,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,0.150812,-0.088923,0.056425,0.996380,...,-0.756332,-1.232434,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625
3141408,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,0.381069,-1.324759,0.056425,-1.111730,...,-0.756332,-1.232434,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040


,target
0,-0.300875
1,-0.231040
2,0.568807
3,-1.064780
4,-0.531940
...,...
3141405,0.033600
3141406,-0.223264
3141407,-0.559415
3141408,0.009599


,time_id
0,0
1,0
2,0
3,0
4,0
...,...
3141405,1219
3141406,1219
3141407,1219
3141408,1219


In [3]:
time_ids = np.sort(time.time_id.unique())
len(time_ids)

1211

In [4]:
n_time_steps = len(time_ids)
print("time steps:", n_time_steps)

valid_prop = 0.2
valid_size = int(valid_prop * n_time_steps)
print("valid size:", valid_size)

time steps: 1211
valid size: 242


In [5]:
valid_time_ids = time_ids[-valid_size:]

In [6]:
train_idx = time.query("time_id not in @valid_time_ids").index
valid_idx = time.query("time_id in @valid_time_ids").index

In [7]:
import gc
gc.collect()

47

***
## Bayesian Optimization

In [8]:
model_params = dict(
    objective="mse",
    metric="rmse",
    num_leaves=32,
    learning_rate=0.05,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    force_col_wise=True,
)

In [9]:
%%time
train_dset = lgb.Dataset(
    data=features.loc[train_idx,:],
    label=target.loc[train_idx,"target"].values,
    free_raw_data=True
)
valid_dset = lgb.Dataset(
    data=features.loc[valid_idx,:],
    label=target.loc[valid_idx,"target"].values,
    free_raw_data=True
)
model = olgb.train(
    params=model_params,
    train_set=train_dset,
    num_boost_round=3000,
    valid_sets=[valid_dset,],
    early_stopping_rounds=50,
    verbose_eval=50,
)

[I 2022-01-31 15:49:52,678] A new study created in memory with name: no-name-f6494c0f-d1c3-40b5-b419-2d320717fc97
feature_fraction, val_score: inf:   0% 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 2364296, number of used features: 300
[LightGBM] [Info] Start training from score -0.024609
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 0.901372
[100]	valid_0's rmse: 0.900459
[150]	valid_0's rmse: 0.899991
[200]	valid_0's rmse: 0.899701
[250]	valid_0's rmse: 0.899467
[300]	valid_0's rmse: 0.899386
[350]	valid_0's rmse: 0.899366
[400]	valid_0's rmse: 0.899278
[450]	valid_0's rmse: 0.899258


feature_fraction, val_score: 0.899245:  14% 1/7 [04:12<25:13, 252.20s/it]

Early stopping, best iteration is:
[415]	valid_0's rmse: 0.899245


[I 2022-01-31 15:54:05,068] Trial 0 finished with value: 0.8992450838377253 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 0.8992450838377253.
feature_fraction, val_score: 0.899245:  14% 1/7 [04:12<25:13, 252.20s/it]/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

Early stopping, best iteration is:
[363]	valid_0's rmse: 0.899005


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 2364296, number of used features: 300
[LightGBM] [Info] Start training from score -0.024609
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 0.900969
[100]	valid_0's rmse: 0.900012
[150]	valid_0's rmse: 0.899466
[200]	valid_0's rmse: 0.899269
[250]	valid_0's rmse: 0.899156
[300]	valid_0's rmse: 0.899083
[350]	valid_0's rmse: 0.899086
[400]	valid_0's rmse: 0.899052


min_data_in_leaf, val_score: 0.898872: 100% 5/5 [15:52<00:00, 191.22s/it][I 2022-01-31 19:06:58,727] Trial 67 finished with value: 0.8990154735279902 and parameters: {'min_child_samples': 50}. Best is trial 63 with value: 0.8988720966666612.
min_data_in_leaf, val_score: 0.898872: 100% 5/5 [15:52<00:00, 190.60s/it]

Early stopping, best iteration is:
[389]	valid_0's rmse: 0.899015
CPU times: user 1d 7h 48min 20s, sys: 2min 21s, total: 1d 7h 50min 41s
Wall time: 3h 17min 9s


In [11]:
model.params

{'objective': 'mse',
 'metric': 'rmse',
 'num_leaves': 55,
 'learning_rate': 0.05,
 'feature_fraction': 0.616,
 'bagging_fraction': 0.8,
 'force_col_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 3.563444727371679,
 'lambda_l2': 0.30026771722107604,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 3000,
 'early_stopping_round': 50}

In [12]:
model.best_iteration

360

In [13]:
model.best_score

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('rmse', 0.8988720966666612)])})

***